In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings

# Gereksiz uyarıları gizle (Temiz çıktı için)
warnings.filterwarnings("ignore")

# Dosya Yükle
DATA_FILE = "train.csv"
print("📊 Model Performans Analizi Başlıyor...\n")

def evaluate():
    # Veriyi Oku ve Hazırla
    df = pd.read_csv(DATA_FILE)
    df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
    
    # Aylık Grupla (Pandas yeni versiyon için 'ME' kullanıyoruz)
    df_monthly = df.groupby(['Category', pd.Grouper(key='Order Date', freq='ME')])['Sales'].sum().reset_index()
    
    categories = df_monthly['Category'].unique()
    
    # Başlıkları yazdır
    print(f"{'KATEGORİ':<20} | {'MAPE (Hata Payı)':<20} | {'DURUM':<10}")
    print("-" * 65)
    
    for cat in categories:
        # Veriyi Al
        cat_data = df_monthly[df_monthly['Category'] == cat].set_index('Order Date')['Sales']
        
        # Frekansı açıkça belirt (Statsmodels uyarısını çözmek için)
        cat_data.index.freq = 'ME'
        
        # Train/Test Ayrımı (Son 12 ayı test için sakla)
        train_data = cat_data.iloc[:-12]
        test_data = cat_data.iloc[-12:]
        
        # Modeli Eğit
        try:
            model = ExponentialSmoothing(
                train_data,
                seasonal_periods=12,
                trend='add',
                seasonal='add'
            ).fit(optimized=True)
            
            # Test Dönemi İçin Tahmin Yap
            predictions = model.forecast(12)
            
            # MAPE Hesapla
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100
            
            # Durum Değerlendirmesi
            status = "🟢 Mükemmel" if mape < 20 else "🟡 İyi" if mape < 40 else "🔴 Geliştirilmeli"
            
            # DÜZELTME BURADA: {mape:<19.2f} (Önce hizalama, sonra ondalık)
            print(f"{cat:<20} | %{mape:<19.2f} | {status}")
            
        except Exception as e:
            print(f"{cat:<20} | Hata: {e}")

    print("\nℹ️ NOT: Perakende satış verileri çok dalgalı olduğu için %30-%40 hata payı (MAPE) endüstride kabul edilebilir standarttır.")

if __name__ == "__main__":
    evaluate()

📊 Model Performans Analizi Başlıyor...

KATEGORİ             | MAPE (Hata Payı)     | DURUM     
-----------------------------------------------------------------
Furniture            | %32.74               | 🟡 İyi
Office Supplies      | %31.97               | 🟡 İyi
Technology           | %40.11               | 🔴 Geliştirilmeli

ℹ️ NOT: Perakende satış verileri çok dalgalı olduğu için %30-%40 hata payı (MAPE) endüstride kabul edilebilir standarttır.
